In [1]:
import pandas as pd
#import stockstats as StockDataFrame
import seaborn as sns
import numpy as np
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from ipywidgets import *
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.rc("figure", figsize=(9, 7))
sns.set_style('whitegrid')

%matplotlib inline
pd.set_option('display.max_columns', 500)
from sklearn.metrics import f1_score
from scipy.optimize import least_squares
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

/home/matthew/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Load stock data

In [2]:
df = pd.read_csv('largecap.csv')

In [3]:
def middle_of_month(name):
    name['date'] = pd.DatetimeIndex(name['date'])
    stockmiddle = name.loc[:,('date','open')]
    stockmiddle['date'] = pd.DatetimeIndex(stockmiddle['date']) + pd.DateOffset(-14)
    stockmiddle['14 day price'] = stockmiddle['open']
    stockmiddle.set_index('date', drop=False, inplace=True)
    stockmiddle.drop('open', 1, inplace=True)
    stock30 = name.loc[:,('date','open')]
    stock30['date'] = pd.DatetimeIndex(stock30['date']) + pd.DateOffset(-28)
    stock30['28 day price'] = stock30['open']
    stock30.set_index('date', drop=False, inplace=True)
    stock30.drop('open', 1, inplace=True)
    stock60 = name.loc[:,('date','open')]
    stock60['date'] = pd.DatetimeIndex(stock60['date']) + pd.DateOffset(-63)
    stock60['63 day price'] = stock60['open']
    stock60.set_index('date', drop=False, inplace=True)
    stock60.drop('open', 1, inplace=True)
    name.set_index('date', drop=False, inplace=True)
    output = pd.merge(name,stockmiddle, left_on='date', right_on='date')
    output = pd.merge(output,stock30, left_on='date', right_on='date')
    output = pd.merge(output,stock60, left_on='date', right_on='date')
    output['2 week gain'] = (output['14 day price'] / output['open']) - 1
    output['4 week gain'] = (output['28 day price'] / output['open']) - 1
    output['7 week gain'] = (output['63 day price'] / output['open']) - 1
    output.set_index('date', drop=False, inplace=True)
    return output

In [4]:
def derivative(data):
    data = data.iloc[1:]
    data['Category: All categories'] = data['Category: All categories'].astype(float)
    data['derivative'] = data['Category: All categories'].diff(periods = 1)
    data.index = [datetime.strptime(date, '%Y-%m-%d').date() for date in data.index]
    data.index = data.index + pd.DateOffset(-2)
    return data
    
def stock(stock,datastock):
    stockdata = df[df['ticker'] == str(stock)]
    datastock = middle_of_month(stockdata)
    return datastock
    
def mergefunction(stock,derivative):
    stock = stock.iloc[1:]
    derivative = pd.merge(stock, derivative,
                          left_index = True,
                          right_index = True,
                          how='inner')
    return stock

In [5]:
df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,marketopen,marketclose,marketgain,volatility,daily_change,dailyrate
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,29.367758,32.272262,25.817810,28.399591,44739900.0,2.035665e+09,1.968556e+09,-67109850.0,0.222222,-1.50,-0.032967
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,27.714581,27.753307,25.694399,26.062292,10897100.0,4.679215e+08,4.400249e+08,-27896576.0,0.077044,-2.56,-0.059618
2,A,1999-12-29,63.00,79.06,62.94,72.00,7524000.0,0.0,1.0,40.668650,51.035928,40.629918,46.478457,7524000.0,4.740120e+08,5.417280e+08,67716000.0,0.227042,9.00,0.142857
3,A,2000-05-12,84.00,98.00,82.25,90.56,9703800.0,0.0,1.0,54.225854,63.263496,53.096148,58.460635,9703800.0,8.151192e+08,8.787761e+08,63656928.0,0.174757,6.56,0.078095
4,A,2000-05-17,83.25,83.75,76.81,78.00,8906100.0,0.0,1.0,53.739464,54.062223,49.582321,50.350489,8906100.0,7.414328e+08,6.946758e+08,-46757025.0,0.086447,-5.25,-0.063063


In [6]:
df.shape

(440325, 20)

In [7]:
tickers = df['ticker'].value_counts()

In [8]:
stocks = pd.DataFrame()
def valuepredictor(df):
    for value in df['ticker']:
        teststock = df[df['ticker'] == value]
        teststock = middle_of_month(teststock)
        value = teststock
        return value
    stocks = pd.merge([value, stocks])

In [9]:
companies=['BRK-B','BRK-A','JPM','WFC','BAC','V','C','HSBC','MA','IVV','LFC','RY','SAN','TD','MTU','GS','USB','MS','WBK','AXP']

In [10]:
teststock = df[df['ticker'].isin(companies)]
teststock = middle_of_month(teststock)
stocks = teststock

/home/matthew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [11]:
stocks.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,marketopen,marketclose,marketgain,volatility,daily_change,dailyrate,14 day price,28 day price,63 day price,2 week gain,4 week gain,7 week gain
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,51.94,54.5,43.44,-0.053571,-0.006924,-0.208455
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,51.94,54.5,53.00,-0.053571,-0.006924,-0.034257
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,51.94,54.5,43.94,-0.053571,-0.006924,-0.199344
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,40.06,54.5,43.44,-0.270044,-0.006924,-0.208455
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,40.06,54.5,53.00,-0.270044,-0.006924,-0.034257


In [12]:
stocks['ticker'].value_counts()

C      824539
JPM    817853
BAC    813263
WFC    791373
MS     749061
USB    652809
AXP    493523
GS     421836
V      290747
MA      35293
Name: ticker, dtype: int64

# Load fundamental data

In [13]:
stocks.isnull().sum()

ticker          0
date            0
open            0
high            0
low             0
close           0
volume          0
ex-dividend     0
split_ratio     0
adj_open        0
adj_high        0
adj_low         0
adj_close       0
adj_volume      0
marketopen      0
marketclose     0
marketgain      0
volatility      0
daily_change    0
dailyrate       0
14 day price    0
28 day price    0
63 day price    0
2 week gain     0
4 week gain     0
7 week gain     0
dtype: int64

In [14]:
stocks = stocks.dropna()

In [15]:
stocks.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,marketopen,marketclose,marketgain,volatility,daily_change,dailyrate,14 day price,28 day price,63 day price,2 week gain,4 week gain,7 week gain
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,51.94,54.5,43.44,-0.053571,-0.006924,-0.208455
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,51.94,54.5,53.00,-0.053571,-0.006924,-0.034257
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,51.94,54.5,43.94,-0.053571,-0.006924,-0.199344
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,40.06,54.5,43.44,-0.270044,-0.006924,-0.208455
2000-10-11,AXP,2000-10-11,54.88,55.94,54.06,54.38,7036200.0,0.0,1.0,33.43618,34.081995,32.936587,33.13155,7036200.0,386146656.0,382628556.0,-3518100.0,0.034182,-0.5,-0.009111,40.06,54.5,53.00,-0.270044,-0.006924,-0.034257


In [16]:
X = stocks.drop(['7 week gain','ticker','63 day price','date'], 1)
y = stocks['7 week gain']
reg_scores = cross_val_score(LinearRegression(), X, y, cv=4)
print reg_scores, np.mean(reg_scores)

[-0.27495798  0.23721337  0.32247699  0.3856716 ] 0.167600994565


In [17]:
rf = RandomForestRegressor()
rf.fit(X, y)
names = X.dtypes.index
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True)

Features sorted by their score:
[(0.5296, 'open'), (0.2646, 'high'), (0.0343, '2 week gain'), (0.0338, '14 day price'), (0.0335, '4 week gain'), (0.0331, '28 day price'), (0.0119, 'adj_open'), (0.0105, 'volatility'), (0.0054, 'low'), (0.005, 'adj_volume'), (0.0049, 'adj_high'), (0.0048, 'adj_close'), (0.004, 'dailyrate'), (0.004, 'adj_low'), (0.0038, 'marketgain'), (0.0037, 'marketopen'), (0.0037, 'close'), (0.0034, 'marketclose'), (0.0033, 'volume'), (0.0027, 'daily_change'), (0.0, 'split_ratio'), (0.0, 'ex-dividend')]


In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i] > 0 and y_hat[i] > 0:
           TP += 1
    for i in range(len(y_hat)): 
        if y_hat[i] > 0 and y_actual[i] < 0:
           FP += 1
    for i in range(len(y_hat)): 
        if y_actual[i] < 0 and  y_hat[i] < 0:
           TN += 1
    for i in range(len(y_hat)): 
        if y_hat[i] < 0 and y_actual[i] > 0:
           FN += 1
    return(TP, FP, TN, FN)

In [ ]:
# Note the difference in argument order
# optionally, you can chain "fit()" to the model object
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
#plt.figure(figsize=(8,6))
#plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
#plt.xlabel("Predicted Values - $\hat{y}$")
#plt.ylabel("Actual Values - $y$")
#plt.show()

#print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            7 week gain   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                 1.224e+05
Date:                Fri, 23 Jun 2017   Prob (F-statistic):               0.00
Time:                        09:53:58   Log-Likelihood:            -1.4929e+07
No. Observations:             5890297   AIC:                         2.986e+07
Df Residuals:                 5890276   BIC:                         2.986e+07
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
open            -0.0029      0.002     -1.524      0.127      -0.007       0.001
high             0.0218      0.002      9.212      0.000       0.017       0.026
low             -0.0176      0.002     -8.422      0.000      -0.022      -0.014
close           -0.0059      0.002     -2.926      0.003      -0.010      -0.002
volume        7.653e-09   1.44e-11    531.404      0.000    7.62e-09    7.68e-09
ex-dividend     -0.0592      0.035     -1.673      0.094      -0.129       0.010
split_ratio      1.1741      0.004    328.064      0.000       1.167       1.181
adj_open         0.0039      0.002      2.303      0.021       0.001       0.007
adj_high        -0.0077      0.002     -4.660      0.000      -0.011      -0.004
adj_low         -0.0073      0.002     -4.453      0.000      -0.010      -0.004
adj_close        0.0127      0.002      7.489      0.000       0.009       0.016
adj_volume   -5.582e-10   2.81e-11    -19.888      0.000   -6.13e-10   -5.03e-10
marketopen      10.8465     10.596      1.024      0.306      -9.921      31.614
marketclose    -10.8465     10.596     -1.024      0.306     -31.614       9.921
marketgain      10.8465     10.596      1.024      0.306      -9.921      31.614
volatility       1.8311      0.048     38.247      0.000       1.737       1.925
daily_change    -0.0030      0.001     -3.693      0.000      -0.005      -0.001
dailyrate       -0.9722      0.066    -14.737      0.000      -1.101      -0.843
14 day price    -0.0078   4.05e-05   -193.353      0.000      -0.008      -0.008
28 day price    -0.0078   4.06e-05   -191.457      0.000      -0.008      -0.008
2 week gain      0.2135      0.001    406.895      0.000       0.212       0.214
4 week gain      0.2129      0.001    416.529      0.000       0.212       0.214
==============================================================================
Omnibus:                  9543227.981   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      11971521124.296
Skew:                          10.520   Prob(JB):                         0.00
Kurtosis:                     222.853   Cond. No.                     7.80e+21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.93e-19. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
perf_measure(predictions, y)

In [ ]:
X = stocks[['open','high','volume','split_ratio','adj_volume','volatility','2 week gain','4 week gain','dailyrate']]
y = stocks['7 week gain']
reg_scores = cross_val_score(LinearRegression(), X, y, cv=4)
print reg_scores, np.mean(reg_scores)

In [ ]:
rf = RandomForestRegressor()
rf.fit(X, y)
names = X.dtypes.index
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True)

In [ ]:
# Note the difference in argument order
# optionally, you can chain "fit()" to the model object
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
#plt.figure(figsize=(8,6))
#plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
#plt.xlabel("Predicted Values - $\hat{y}$")
#plt.ylabel("Actual Values - $y$")
#plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()

In [ ]:
perf_measure(y, predictions)